In [21]:
import os 


with open("models/extract_prism_data.P", "r") as yw_struct:
    line = yw_struct.readline()
    while line:
        print(line, end='')
        line = yw_struct.readline()
  


% FACT: program(program_id, program_name, qualified_program_name, begin_annotation_id, end_annotation_id).
program(1, 'extract_prism_data', 'extract_prism_data', 1, 12).
program(2, 'extract_prism_data', 'extract_prism_data.extract_prism_data', 1, 12).

% FACT: workflow(program_id).
workflow(1).

% FACT: function(program_id).

% FACT: has_subprogram(program_id, subprogram_id).
has_subprogram(1, 2).

% FACT: port(port_id, port_type, port_name, qualified_port_name, port_annotation_id, data_id).
port(1, 'in', 'coordinates', 'extract_prism_data<-coordinates', 3, 1).
port(2, 'param', 'prism_data', 'extract_prism_data<-prism_data', 5, 2).
port(3, 'out', 'prism_data_for_coordinates', 'extract_prism_data->prism_data_for_coordinates', 9, 3).
port(4, 'in', 'coordinates', 'extract_prism_data.extract_prism_data<-coordinates', 3, 1).
port(5, 'param', 'prism_data', 'extract_prism_data.extract_prism_data<-prism_data', 5, 2).
port(6, 'out', 'prism_data_for_coordinates', 'extract_prism_data.extract_pri

In [20]:
file_data.

['\n',
 '% FACT: program(program_id, program_name, qualified_program_name, begin_annotation_id, end_annotation_id).\n',
 "program(1, 'extract_prism_data', 'extract_prism_data', 1, 12).\n",
 "program(2, 'extract_prism_data', 'extract_prism_data.extract_prism_data', 1, 12).\n",
 '\n',
 '% FACT: workflow(program_id).\n',
 'workflow(1).\n',
 '\n',
 '% FACT: function(program_id).\n',
 '\n',
 '% FACT: has_subprogram(program_id, subprogram_id).\n',
 'has_subprogram(1, 2).\n',
 '\n',
 '% FACT: port(port_id, port_type, port_name, qualified_port_name, port_annotation_id, data_id).\n',
 "port(1, 'in', 'coordinates', 'extract_prism_data<-coordinates', 3, 1).\n",
 "port(2, 'param', 'prism_data', 'extract_prism_data<-prism_data', 5, 2).\n",
 "port(3, 'out', 'prism_data_for_coordinates', 'extract_prism_data->prism_data_for_coordinates', 9, 3).\n",
 "port(4, 'in', 'coordinates', 'extract_prism_data.extract_prism_data<-coordinates', 3, 1).\n",
 "port(5, 'param', 'prism_data', 'extract_prism_data.extrac